In [1]:
print("strawberry")

strawberry


In [2]:
import torch
import numpy as np

# dataset
from twaidata.torchdatasets.in_ram_ds import MRISegmentation2DDataset, MRISegmentation3DDataset
from torch.utils.data import DataLoader, random_split, ConcatDataset

# model
from trustworthai.models.uq_models.drop_UNet import UNet
from trustworthai.models.uq_models.HyperMapp3r import HyperMapp3r
from trustworthai.models.base_models.torchUNet import UNet as StandardUNet

# augmentation and pretrain processing
from trustworthai.utils.augmentation.standard_transforms import RandomFlip, GaussianBlur, GaussianNoise, \
                                                            RandomResizeCrop, RandomAffine, \
                                                            NormalizeImg, PairedCompose, LabelSelect, \
                                                            PairedCentreCrop, CropZDim
# loss function
from trustworthai.utils.losses_and_metrics.tversky_loss import TverskyLoss
from trustworthai.utils.losses_and_metrics.misc_metrics import IOU
from trustworthai.utils.losses_and_metrics.dice import dice, DiceMetric
from trustworthai.utils.losses_and_metrics.dice_losses import DiceLoss, GeneralizedDiceLoss
from trustworthai.utils.losses_and_metrics.power_jaccard_loss import PowerJaccardLoss
from torch.nn import BCELoss, MSELoss, BCEWithLogitsLoss

# fitter
from trustworthai.utils.fitting_and_inference.fitters.basic_lightning_fitter import StandardLitModelWrapper
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import pytorch_lightning as pl

# misc
import os
import torch
import matplotlib.pyplot as plt
import torch
from torchinfo import summary

### Set the seed

In [3]:
seed = 3407
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

### define datasets and dataloaders

In [4]:
is3D = False

In [5]:
root_dir = "/disk/scratch/s2208943/ipdis/preprep/out_data/collated/"
wmh_dir = root_dir + "WMH_challenge_dataset/"
ed_dir = root_dir + "EdData/"

In [6]:
# domains = [
#             wmh_dir + d for d in ["Singapore", "Utrecht", "GE3T"]
#           ]

# domains = [
#             wmh_dir + d for d in ["Singapore", "Utrecht", "GE3T"]
#           ] + [
#             ed_dir + d for d in ["domainA", "domainB", "domainC", "domainD"]
#           ]


domains = [
            ed_dir + d for d in ["domainA", "domainB", "domainC", "domainD"]
          ]

In [7]:
# augmentation definintion
def get_transforms(is_3D):
    transforms = [
        LabelSelect(label_id=1),
        RandomFlip(p=0.5, orientation="horizontal"),
        # GaussianBlur(p=0.5, kernel_size=7, sigma=(.1, 1.5)),
        # GaussianNoise(p=0.2, mean=0, sigma=0.2),
        # RandomAffine(p=0.2, shear=(.1,3.)),
        # RandomAffine(p=0.2, degrees=5),
        # RandomResizeCrop(p=1., scale=(0.6, 1.), ratio=(3./4., 4./3.))
    ]
    if not is_3D:
        return PairedCompose(transforms)
    else:
        transforms.append(CropZDim(size=32, minimum=0, maximum=-1))
        return PairedCompose(transforms)

In [8]:
# function to do train validate test split
test_proportion = 0.1
validation_proportion = 0.2

def train_val_test_split(dataset, val_prop, test_prop, seed):
    # I think the sklearn version might be prefereable for determinism and things
    # but that involves fiddling with the dataset implementation I think....
    size = len(dataset)
    test_size = int(test_prop*size) 
    val_size = int(val_prop*size)
    train_size = size - val_size - test_size
    train, val, test = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(seed))
    return train, val, test

In [9]:
# load datasets
# this step is quite slow, all the data is being loaded into memory
if is3D:
    datasets_domains = [MRISegmentation3DDataset(root_dir, domain, transforms=get_transforms(is_3D=True)) for domain in domains]
else:
    datasets_domains = [MRISegmentation2DDataset(root_dir, domain, transforms=get_transforms(is_3D=False)) for domain in domains]

# split into train, val test datasets
datasets = [train_val_test_split(dataset, validation_proportion, test_proportion, seed) for dataset in datasets_domains]

# concat the train val test datsets
train_dataset = ConcatDataset([ds[0] for ds in datasets])
val_dataset = ConcatDataset([ds[1] for ds in datasets])
test_dataset = ConcatDataset([ds[2] for ds in datasets])

In [10]:
len(train_dataset), len(val_dataset), len(test_dataset)

(8743, 2497, 1248)

In [11]:
# define dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size = 16, shuffle=False, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

### setup model

In [61]:
in_channels = 3
out_channels = 1

if is3D:
    model = UNet3D(in_channels,
                 out_channels,
                 init_features=32,
                 kernel_size=3,
                 softmax=False,
                 dropout_type=None,
                 dropout_p=None,
                 gaussout_mean=None, 
                 dropconnect_type=None,
                 dropconnect_p=None,
                 gaussconnect_mean=None,
                 norm_type="bn", 
                 use_multidim_dropout = None,  
                 use_multidim_dropconnect = None, 
                 groups=None,
                 gn_groups=None, 
                )
    optimizer_params={"lr":2e-3}
    lr_scheduler_params={"step_size":100, "gamma":0.5}
else:
    # model = UNet(in_channels,
    #              out_channels,
    #              kernel_size=3,
    #              init_features=32,
    #              softmax=False,
    #              dropout_type="bernoulli",
    #              dropout_p=0.1,
    #              gaussout_mean=None, 
    #              dropconnect_type="gaussian",
    #              dropconnect_p=0.1,
    #              gaussconnect_mean=1,
    #              norm_type="bn", 
    #              use_multidim_dropout = True,  
    #              use_multidim_dropconnect = True, 
    #              groups=None,
    #              gn_groups=None, 
    #             )
    # optimizer_params={"lr":1e-3}
    # lr_scheduler_params={"step_size":30, "gamma":0.1}
    encoder_features=[16, 32, 64, 128, 256] # orig: [16, 32, 64, 128, 256]
    decoder_features=encoder_features[::-1][1:]
    model = HyperMapp3r(dims=2, in_channels=in_channels, out_channels=1, softmax=False,
                       encoder_features=encoder_features,
                 decoder_features=decoder_features)
    optimizer_params={"lr":1e-3}
    lr_scheduler_params={"step_size":5, "gamma":0.1}


In [62]:
print("WARNING: FOR SOME REASON PARAMETERS INSIDE DROPCONNECT BLOCKS ARE NOT REGISTERING THEIR NUMBER OF PARAMS CORRECTLY")

In [63]:
summary(model, (1, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
HyperMapp3r                              --                        --
├─ModuleList: 1-1                        --                        --
├─ModuleList: 1-2                        --                        --
├─ModuleList: 1-3                        --                        --
├─ModuleList: 1-4                        --                        --
├─ModuleList: 1-5                        --                        --
├─ModuleList: 1-2                        --                        --
│    └─Conv2d: 2-1                       [1, 16, 64, 64]           448
├─ModuleList: 1-1                        --                        --
│    └─HM3Block: 2-2                     [1, 16, 64, 64]           --
│    │    └─InstanceNorm2d: 3-1          [1, 16, 64, 64]           --
│    │    └─LeakyReLU: 3-2               [1, 16, 64, 64]           --
│    │    └─Conv2d: 3-3                  [1, 16, 64, 64]           2,304
│    │    └

In [64]:
loss = GeneralizedDiceLoss(normalization='sigmoid')

In [65]:
model = StandardLitModelWrapper(model, loss, 
                                optimizer_params=optimizer_params,
                                lr_scheduler_params=lr_scheduler_params,
                                is_uq_model=True
                               )

In [66]:
checkpoint_dir = "/disk/scratch/s2208943/results/"
strategy = None
# strategy = "deepspeed_stage_2"
# strategy = "dp"
#strategy = "deepspeed_stage_2_offload"

accelerator="gpu"
devices=1
max_epochs=1000
precision = 16

checkpoint_callback = ModelCheckpoint(checkpoint_dir, save_top_k=2, monitor="val_loss")
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=100, verbose="False", mode="min", check_finite=True)
trainer = pl.Trainer(
    callbacks=[checkpoint_callback, early_stop_callback],
    accelerator=accelerator,
    devices=devices,
    max_epochs=max_epochs,
    strategy=strategy,
    precision=precision,
    default_root_dir=checkpoint_dir
)


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### train

In [67]:
trainer.fit(model, train_dataloader, val_dataloader)

/home/s2208943/miniconda3/envs/ip/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /disk/scratch/s2208943/results exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                | Params
--------------------------------------------------------
0 | model           | HyperMapp3r         | 2.8 M 
1 | loss            | GeneralizedDiceLoss | 0     
2 | logging_metrics | ModuleList          | 0     
--------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
5.549     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.353


...

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.249


Validation: 0it [00:00, ?it/s]

Metric val_loss improved by 0.000 >= min_delta = 0.0. New best score: 0.249


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.validate(model, val_dataloader, ckpt_path="best")

In [69]:
trainer.test(model, test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           0.2667236924171448
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.2667236924171448}]